In [1]:
# Import dependcies
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier

# Create global seed
yogi = 8

In [2]:
# Read in data 
file_path = '../Data/Pitchers/kershaw.csv'
kershaw_df = pd.read_csv(file_path)

kershaw_df.dropna(inplace = True)

kershaw_df.head(10)

,Unnamed: 0,pitch_type,release_speed,release_pos_x,release_pos_z,player_name,zone,game_type,stand,p_throws,...,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name,spin_axis,delta_home_win_exp,delta_run_exp
0,11575,FF,90.0,1.26,6.16,"Kershaw, Clayton",4.0,R,R,L,...,90.0,2526.0,6.3,54.23,34,4,4-Seam Fastball,176.0,0.028,-0.355
1,11576,FF,90.4,1.43,6.13,"Kershaw, Clayton",6.0,R,R,L,...,90.4,2454.0,6.3,54.25,34,3,4-Seam Fastball,174.0,0.000,-0.078
2,11577,SL,88.0,1.38,6.12,"Kershaw, Clayton",13.0,R,R,L,...,87.9,2665.0,6.3,54.23,34,2,Slider,199.0,0.000,0.061
3,11578,FF,90.2,1.42,6.09,"Kershaw, Clayton",14.0,R,R,L,...,90.3,2498.0,6.5,54.04,34,1,4-Seam Fastball,177.0,0.000,0.040
4,11579,CU,72.9,1.01,6.40,"Kershaw, Clayton",6.0,R,R,L,...,72.6,2496.0,6.3,54.24,33,8,Curveball,342.0,0.026,-0.286
5,11580,SL,86.4,1.39,6.23,"Kershaw, Clayton",5.0,R,R,L,...,86.7,2560.0,6.3,54.21,33,7,Slider,220.0,0.000,0.000
6,11581,FF,90.5,1.33,6.20,"Kershaw, Clayton",5.0,R,R,L,...,90.3,2412.0,6.2,54.34,33,6,4-Seam Fastball,174.0,0.000,0.000
7,11582,SL,86.4,1.29,6.21,"Kershaw, Clayton",13.0,R,R,L,...,86.4,2651.0,6.3,54.22,33,5,Slider,213.0,0.000,0.026
8,11583,CU,72.8,1.02,6.32,"Kershaw, Clayton",13.0,R,R,L,...,72.6,2632.0,6.3,54.23,33,4,Curveball,340.0,0.000,0.000
9,11584,SL,86.4,1.33,6.19,"Kershaw, Clayton",13.0,R,R,L,...,86.8,2692.0,6.4,54.09,33,3,Slider,204.0,0.000,0.003


In [3]:
# Drop Unnamed columns
kershaw_df.drop(['Unnamed: 0', 'pitch_name'], axis = 1, inplace = True)

In [4]:
# Generate categorical variable list
pitch_cat = kershaw_df.dtypes[kershaw_df.dtypes == 'object'].index.tolist()
pitch_cat.remove('type')
pitch_cat.remove('player_name')
pitch_cat

['pitch_type', 'game_type', 'stand', 'p_throws']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(kershaw_df[pitch_cat]))

# Add the encoded varibale names to the DataFrame
encode_df.columns = enc.get_feature_names(pitch_cat)
encode_df.head()

,pitch_type_CH,pitch_type_CU,pitch_type_FF,pitch_type_SL,game_type_R,stand_L,stand_R,p_throws_L
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
2,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
3,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0


In [6]:
# Merge one-hot encoded features and drop the originals
kershaw_df = kershaw_df.merge(
    encode_df,
    left_index = True,
    right_index = True
).drop(pitch_cat, 1)

kershaw_df.head()

,release_speed,release_pos_x,release_pos_z,player_name,zone,type,balls,strikes,pfx_x,pfx_z,...,delta_home_win_exp,delta_run_exp,pitch_type_CH,pitch_type_CU,pitch_type_FF,pitch_type_SL,game_type_R,stand_L,stand_R,p_throws_L
0,90.0,1.26,6.16,"Kershaw, Clayton",4.0,X,2,1,-0.32,1.59,...,0.028,-0.355,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
1,90.4,1.43,6.13,"Kershaw, Clayton",6.0,S,2,0,-0.20,1.49,...,0.000,-0.078,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
2,88.0,1.38,6.12,"Kershaw, Clayton",13.0,B,1,0,-0.43,0.74,...,0.000,0.061,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
3,90.2,1.42,6.09,"Kershaw, Clayton",14.0,B,0,0,-0.40,1.65,...,0.000,0.040,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
4,72.9,1.01,6.40,"Kershaw, Clayton",6.0,S,2,2,-0.20,-1.44,...,0.026,-0.286,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0


In [7]:
# Drop superfluous columns 
kershaw_df.drop(columns = ['stand_L'], axis = 1, inplace = True)
kershaw_df.head()


,release_speed,release_pos_x,release_pos_z,player_name,zone,type,balls,strikes,pfx_x,pfx_z,...,spin_axis,delta_home_win_exp,delta_run_exp,pitch_type_CH,pitch_type_CU,pitch_type_FF,pitch_type_SL,game_type_R,stand_R,p_throws_L
0,90.0,1.26,6.16,"Kershaw, Clayton",4.0,X,2,1,-0.32,1.59,...,176.0,0.028,-0.355,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,90.4,1.43,6.13,"Kershaw, Clayton",6.0,S,2,0,-0.20,1.49,...,174.0,0.000,-0.078,0.0,0.0,1.0,0.0,1.0,1.0,1.0
2,88.0,1.38,6.12,"Kershaw, Clayton",13.0,B,1,0,-0.43,0.74,...,199.0,0.000,0.061,0.0,0.0,0.0,1.0,1.0,1.0,1.0
3,90.2,1.42,6.09,"Kershaw, Clayton",14.0,B,0,0,-0.40,1.65,...,177.0,0.000,0.040,0.0,0.0,1.0,0.0,1.0,1.0,1.0
4,72.9,1.01,6.40,"Kershaw, Clayton",6.0,S,2,2,-0.20,-1.44,...,342.0,0.026,-0.286,0.0,1.0,0.0,0.0,1.0,1.0,1.0


### Compiling, Training, and Testing Data

In [8]:
# Define features set
X = kershaw_df.copy()
X.drop(columns = ['player_name', 'type', 'balls', 'strikes', 'delta_run_exp', 'delta_home_win_exp'], axis = 1, inplace = True)
X.head()

,release_speed,release_pos_x,release_pos_z,zone,pfx_x,pfx_z,plate_x,plate_z,inning,vx0,...,at_bat_number,pitch_number,spin_axis,pitch_type_CH,pitch_type_CU,pitch_type_FF,pitch_type_SL,game_type_R,stand_R,p_throws_L
0,90.0,1.26,6.16,4.0,-0.32,1.59,-0.56,2.35,4,-3.856777,...,34,4,176.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,90.4,1.43,6.13,6.0,-0.20,1.49,0.63,2.66,4,-1.589323,...,34,3,174.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
2,88.0,1.38,6.12,13.0,-0.43,0.74,-0.83,0.07,4,-4.447718,...,34,2,199.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
3,90.2,1.42,6.09,14.0,-0.40,1.65,0.25,1.06,4,-2.049258,...,34,1,177.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
4,72.9,1.01,6.40,6.0,-0.20,-1.44,0.68,2.18,4,-0.318159,...,33,8,342.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0


In [9]:
# Define target vector
y = kershaw_df['type'].values
y[:5]

array(['X', 'S', 'B', 'B', 'S'], dtype=object)

In [10]:
# Split the data into training and testing sets - stratify by pitcher
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = yogi)

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit Standard Scaler 
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Find Best Learning Rate

In [12]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
best_rate = 0
best_acc = 0

for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(
        n_estimators = 20,
        learning_rate = learning_rate,
        max_features = 7,
        max_depth = 4,
        random_state = yogi)
    
    # Fit the model
    classifier.fit(X_train_scaled, y_train)
    print(f'Learning Rate: {learning_rate}')

    # Score the model
    print('Accuracy Score (training): {0:.3f}'.format(
        classifier.score(
            X_train_scaled,
            y_train
        )
    ))
    print('Accuracy Score (validation): {0:.3f}'.format(
        classifier.score(
            X_test_scaled,
            y_test
        )
    ))
    print()

    if classifier.score(X_test_scaled, y_test) > best_acc:
        best_acc = classifier.score(X_test_scaled, y_test)
        best_rate = learning_rate

Learning Rate: 0.05
Accuracy Score (training): 0.718
Accuracy Score (validation): 0.711

Learning Rate: 0.1
Accuracy Score (training): 0.739
Accuracy Score (validation): 0.714

Learning Rate: 0.25
Accuracy Score (training): 0.808
Accuracy Score (validation): 0.698

Learning Rate: 0.5
Accuracy Score (training): 0.883
Accuracy Score (validation): 0.673

Learning Rate: 0.75
Accuracy Score (training): 0.930
Accuracy Score (validation): 0.648

Learning Rate: 1
Accuracy Score (training): 0.942
Accuracy Score (validation): 0.660



### Create model with best learning rate

In [13]:
# Choose a learning rate and create classifiers
classifier = GradientBoostingClassifier(
    n_estimators = 20,
    learning_rate = best_rate,
    max_features = 7,
    max_depth = 3,
    random_state = yogi
)

# Fit the model
classifier.fit(X_train_scaled, y_train)

# Make predictions
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({'Prediction': predictions, 'Actual': y_test}).head(20)

,Prediction,Actual
0,B,B
1,S,S
2,B,B
3,S,B
4,S,X
5,S,S
6,S,S
7,B,S
8,S,X
9,S,S


### Evaluate Model

In [14]:
# Calculate accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.7043147208121827


In [15]:
# Generate classification report
print('Classification Report')
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           B       0.77      0.81      0.79       267
           S       0.67      0.86      0.75       395
           X       0.00      0.00      0.00       126

    accuracy                           0.70       788
   macro avg       0.48      0.56      0.51       788
weighted avg       0.60      0.70      0.64       788



C:\Users\Matthew\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Matthew\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Matthew\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
